In [ ]:
#mounting drive
import pandas as pd
import os
import glob
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
#merge all the reviews
import os
import glob
import pandas as pd

folder_path = '/content/drive/My Drive/reviews of all models'

file_pattern = os.path.join(folder_path, '*.csv')
all_csv_files = glob.glob(file_pattern)

all_dfs_list = []
print(f"--- Found {len(all_csv_files)} starting to process... ---")

for filename in all_csv_files:
    try:
        df = pd.read_csv(filename)
        #(e.g., 'iphone_12_pro.csv') to readable model name ('iphone 12 pro')
        model_name = os.path.basename(filename).replace('.csv', '').replace('_', ' ')
        df['model'] = model_name
        all_dfs_list.append(df)
        print(f"Processed {os.path.basename(filename)}")
    except Exception as e:
        print(f"An error occurred with {os.path.basename(filename)}: {e}")

if all_dfs_list:
    # Concatenate all DataFrames into one
    merged_df = pd.concat(all_dfs_list, ignore_index=True)

    print("\n" + "="*50)
    print("--- All files have been successfully merged! ---")

    # Show summary of merged dataset
    print(f"\nFinal dataset contains {merged_df.shape[0]} reviews from {len(all_dfs_list)} files.")
    print("First 5 rows:")
    print(merged_df.head())

    # Save merged data to a new CSV
    output_filename = 'merged_reviews.csv'
    merged_df.to_csv(output_filename, index=False)

    print(f"\nMerged data saved to '{output_filename}' in your Colab session.")
    print("You can now download this file from the file pane on the left.")
else:
    print("\nNo CSV files were found or processed. Please check the folder path and ensure it contains CSV files.")


In [ ]:
df=pd.read_csv( '/content/merged_reviews.csv')
df.head()

In [ ]:
df.columns


In [ ]:
df.shape


In [ ]:
df.info()

In [ ]:
df['model'].value_counts()

In [ ]:
df['input']

In [ ]:
df['isAmazonVine']

In [ ]:
df['position']

In [ ]:
df['reviewReaction']

In [ ]:
new_df = df[[
    'model',
    'ratingScore',
    'reviewTitle',
    'reviewDescription',
    'date',
]].copy()
print(new_df.head())

In [ ]:
new_df.shape

In [ ]:
new_df.info()

In [ ]:
new_df['date'] = pd.to_datetime(new_df['date'])

In [ ]:
new_df.info()

In [ ]:
new_df['reviewDescription'] = new_df['reviewDescription'].fillna('  ')

In [ ]:
new_df.info()

In [ ]:

#detecting and then translating the spanish reviews
!pip install langdetect googletrans==4.0.0-rc1


from langdetect import detect, LangDetectException
from googletrans import Translator
from tqdm.notebook import tqdm
import pandas as pd

# --- Create the combined text column ---
new_df['full_review_text'] = new_df['reviewTitle'].astype(str) + ". " + new_df['reviewDescription'].astype(str)

# --- Define the language detection function ---
def is_spanish(text):
    try:
        return detect(text) == 'es'
    except LangDetectException:
        return False

# --- Identify Spanish reviews ---
print("Detecting language for each review...")
tqdm.pandas(desc="Language Detection")
is_spanish_series = new_df['full_review_text'].progress_apply(is_spanish)
spanish_indices = is_spanish_series[is_spanish_series == True].index
print(f" Found {len(spanish_indices)} Spanish reviews to translate.")

# --- Translate and overwrite in the same column ---
translator = Translator()

# Loop only through the reviews identified as Spanish
for i in tqdm(spanish_indices, desc="Translating Spanish Reviews"):
    try:
        original_text = new_df.loc[i, 'full_review_text']
        translated_text = translator.translate(original_text, src='es', dest='en').text
        # Overwrite the original text with the translation
        new_df.loc[i, 'full_review_text'] = translated_text
    except Exception as e:
        print(f"Could not translate row {i}: {e}")

print("\n In-place translation complete.")
print("\n--- DataFrame with Updated 'full_review_text' Column ---")
print(new_df[['full_review_text']].head())

In [ ]:
#applying RoBERTa
from transformers import pipeline
from tqdm import tqdm #this is just to track progress
tqdm.pandas()

sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")


In [ ]:
new_df['full_review_text']

In [ ]:
#sentiment analysis
def get_sentiment(text):
    try:
        truncated_text = ' '.join(str(text).split()[:510])
        result = sentiment_pipeline(truncated_text)[0]
        if result['label'] == 'Positive':
            return 1
        elif result['label'] == 'Neutral':
            return 0
        else:
            return -1
    except Exception:
        return 0

new_df['sentiment_score'] = new_df['full_review_text'].progress_apply(get_sentiment)

In [ ]:
negative_sentiment_count = new_df[new_df['sentiment_score'] == -1].shape[0]

In [ ]:
print(negative_sentiment_count)

In [ ]:
#importing libraries for nltk
!pip install transformers torch
!pip install gensim nltk

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [ ]:
# adding some custom stop words that are common in phone reviews but not very informative
custom_stop_words = ['phone', 'iphone', 'apple', 'amazon', 'product', 'get', 'one',
                     'would', 'like', 'bought', 'buy', 'also', 'issue', 'problem']

stop_words.update(custom_stop_words)

In [ ]:
new_df.shape

In [ ]:
new_df.columns

In [ ]:
#Define a preprocessing pipeline: lowercase, remove punctuation, stopwords, and lemmatize words
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    tokens = text.split()
    processed_tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word not in stop_words and len(word) > 2
    ]
    return processed_tokens

new_df['processed_text'] = new_df['full_review_text'].apply(preprocess_text)

print("Text preprocessing complete.")
print(new_df[['full_review_text', 'processed_text']].head())

In [ ]:
new_df['processed_text']

In [ ]:
#creating bigrams
from gensim.models.phrases import Phrases, Phraser

tokenized_reviews = new_df['processed_text'].tolist()

phrases = Phrases(tokenized_reviews, min_count=5, threshold=10)
bigram_phraser = Phraser(phrases)

new_df.loc[:, 'processed_text_bigrams'] = new_df['processed_text'].apply(lambda tokens: bigram_phraser[tokens])
print("--- Example of bigram creation ---")
print(new_df[['processed_text', 'processed_text_bigrams']].head())


In [ ]:
# Prepare data for LDA: create tokens and dictionary from preprocessed text
# Create a corpus and dictionary from the tokenized reviews
from gensim import corpora

processed_docs = new_df['processed_text_bigrams'].tolist()

dictionary = corpora.Dictionary(processed_docs)

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

print(f"Created a dictionary with {len(dictionary)} unique words.")
print(f"Created a corpus with {len(corpus)} documents.")

In [ ]:


import matplotlib.pyplot as plt
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from tqdm.notebook import tqdm

def find_optimal_topics(dictionary, corpus, texts, limit=16):
    coherence_values = []
    model_list = []

    for num_topics in tqdm(range(2, limit), desc="Finding Optimal Topics"):
        model = LdaMulticore(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            random_state=100,
            chunksize=100,
            passes=10,
            per_word_topics=True
        )
        model_list.append(model)
        coherencemodel = CoherenceModel(
            model=model,
            texts=texts,
            dictionary=dictionary,
            coherence='c_v'
        )
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

model_list, coherence_values = find_optimal_topics(
    dictionary=dictionary,
    corpus=corpus,
    texts=new_df['processed_text_bigrams'].tolist(),
    limit=16
)

x = range(2, 16)
plt.figure(figsize=(10, 6))
plt.plot(x, coherence_values)
plt.xlabel("Number of Topics")
plt.ylabel("Coherence Score")
plt.title("Finding the Optimal Number of Topics")
plt.xticks(x)
plt.grid(True)
plt.show()

In [ ]:
# Build and train the LDA model with 6 topics
import gensim
from gensim.models import CoherenceModel
from pprint import pprint
from tqdm.notebook import tqdm

lda_model_4_topics = gensim.models.LdaMulticore(
    corpus=corpus, id2word=dictionary, num_topics=4,
    random_state=100, chunksize=100, passes=20
)

lda_model_6_topics = gensim.models.LdaMulticore(
    corpus=corpus, id2word=dictionary, num_topics=6,
    random_state=100, chunksize=100, passes=20
)

print("Models trained successfully.")

coherence_4 = CoherenceModel(
    model=lda_model_4_topics,
    texts=new_df['processed_text_bigrams'],
    dictionary=dictionary,
    coherence='c_v'
).get_coherence()

coherence_6 = CoherenceModel(
    model=lda_model_6_topics,
    texts=new_df['processed_text_bigrams'],
    dictionary=dictionary,
    coherence='c_v'
).get_coherence()

print("\nModel Coherence Scores")
print(f"4 Topics: {coherence_4:.4f}")
print(f"6 Topics: {coherence_6:.4f}")

def assign_topics(lda_model, corpus):
    return [max(prob_dist, key=lambda item: item[1])[0]
            for prob_dist in tqdm(lda_model[corpus], desc=f"Assigning {lda_model.num_topics} Topics")]

new_df['topic_4'] = assign_topics(lda_model_4_topics, corpus)
new_df['topic_6'] = assign_topics(lda_model_6_topics, corpus)

print("\nTopic labels assigned.")

print("\nTopics from 4-Topic Model")
pprint(lda_model_4_topics.print_topics())

print("\nTopics from 6-Topic Model")
pprint(lda_model_6_topics.print_topics())

print("\nSample with Topic Labels")
print(new_df[['full_review_text', 'topic_4', 'topic_6']].head())

In [ ]:
# Print out the top words for each discovered topic
import gensim
from gensim.models import CoherenceModel
from pprint import pprint
from tqdm.notebook import tqdm

# Train models
lda_model_2_topics = gensim.models.LdaMulticore(
    corpus=corpus, id2word=dictionary, num_topics=2,
    random_state=100, chunksize=100, passes=20
)

lda_model_3_topics = gensim.models.LdaMulticore(
    corpus=corpus, id2word=dictionary, num_topics=3,
    random_state=100, chunksize=100, passes=20
)

lda_model_5_topics = gensim.models.LdaMulticore(
    corpus=corpus, id2word=dictionary, num_topics=5,
    random_state=100, chunksize=100, passes=20
)

print("Models trained successfully.")

# Coherence scores
coherence_2 = CoherenceModel(
    model=lda_model_2_topics,
    texts=new_df['processed_text_bigrams'],
    dictionary=dictionary,
    coherence='c_v'
).get_coherence()

coherence_3 = CoherenceModel(
    model=lda_model_3_topics,
    texts=new_df['processed_text_bigrams'],
    dictionary=dictionary,
    coherence='c_v'
).get_coherence()

coherence_5 = CoherenceModel(
    model=lda_model_5_topics,
    texts=new_df['processed_text_bigrams'],
    dictionary=dictionary,
    coherence='c_v'
).get_coherence()

print("\nModel Coherence Scores")
print(f"2 Topics: {coherence_2:.4f}")
print(f"3 Topics: {coherence_3:.4f}")
print(f"5 Topics: {coherence_5:.4f}")

def assign_topics(lda_model, corpus):
    return [max(prob_dist, key=lambda item: item[1])[0]
            for prob_dist in tqdm(lda_model[corpus], desc=f"Assigning {lda_model.num_topics} Topics")]

# Assign topic labels
new_df['topic_2'] = assign_topics(lda_model_2_topics, corpus)
new_df['topic_3'] = assign_topics(lda_model_3_topics, corpus)
new_df['topic_5'] = assign_topics(lda_model_5_topics, corpus)

print("\nTopic labels assigned.")

# Output topic summaries
print("\nTopics from 2-Topic Model")
pprint(lda_model_2_topics.print_topics())

print("\nTopics from 3-Topic Model")
pprint(lda_model_3_topics.print_topics())

print("\nTopics from 5-Topic Model")
pprint(lda_model_5_topics.print_topics())

print("\nSample with Topic Labels")
print(new_df[['full_review_text', 'topic_2', 'topic_3', 'topic_5']].head())

In [ ]:
final_topic_map = {
    0: 'Seller & Carrier Unlockability',
    1: 'Phone Functionality & Defects',
    2: 'Screen & Cosmetic Condition',
    3: 'Battery Life & Charging'
}

new_df['final_topic'] = new_df['topic_4'].map(final_topic_map)

print("--- DataFrame with Final, Descriptive Topic Labels ---")
print(new_df[['full_review_text', 'final_topic']].head(20))

In [ ]:
final_topic_distribution = new_df['final_topic'].value_counts(normalize=True) * 100

print("\n--- Final Distribution of Complaint Topics ---")
print(final_topic_distribution)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Plot bar chart of number of reviews per LDA topic
plt.figure(figsize=(12, 7))
sns.barplot(x=final_topic_distribution.values, y=final_topic_distribution.index, palette='viridis')

plt.title('Main Complaint Topics for Renewed iPhones', fontsize=16)
plt.xlabel('Percentage of Reviews (%)', fontsize=12)
plt.ylabel('Complaint Topic', fontsize=12)
plt.show()

In [ ]:
print(new_df.columns.tolist())

In [ ]:
## Generate and display a word cloud of common words across all reviews
!pip install wordcloud --quiet
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Group by final topic
topics = new_df['final_topic'].unique()

for topic in topics:
    text = ' '.join(new_df[new_df['final_topic'] == topic]['full_review_text'].dropna().astype(str))
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"WordCloud for Topic: {topic}", fontsize=16)
    plt.show()

In [ ]:
topics = new_df['final_topic'].unique()

for topic in topics:
    text = ' '.join(new_df[new_df['final_topic'] == topic]['processed_text'].dropna().astype(str))
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f"WordCloud for Topic: {topic}", fontsize=16)
    plt.show()

In [ ]:
# Ensure 'date' is datetime
new_df['date'] = pd.to_datetime(new_df['date'], errors='coerce')

# Drop null dates
timeline_df = new_df.dropna(subset=['date'])

# Plot number of reviews per topic over time
plt.figure(figsize=(14, 7))
for topic in new_df['final_topic'].unique():
    timeline_df[timeline_df['final_topic'] == topic].resample('M', on='date').size().plot(label=topic)

plt.title("Topic Trend Over Time")
plt.xlabel("Month")
plt.ylabel("Number of Reviews")
plt.legend()
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(12, 6))
sns.boxplot(data=new_df, x='final_topic', y='ratingScore')
plt.title("Review Ratings per Topic")
plt.xticks(rotation=45)
plt.show()


In [ ]:
new_df.columns

In [ ]:
new_df.head()

In [ ]:
count_non_negative = (new_df['sentiment_score'] != -1).sum()
print(f"Number of entries where sentiment_score is not -1: {count_non_negative}")

In [ ]:
sentiment_counts = new_df['sentiment_score'].value_counts().sort_index()

print("Sentiment Summary")
print(f"Negative (-1): {sentiment_counts.get(-1, 0)}")
print(f"Neutral (0):   {sentiment_counts.get(0, 0)}")
print(f"Positive (1):  {sentiment_counts.get(1, 0)}")


In [ ]:
columns_for_dashboard = [
    'model',
    'ratingScore',
    'date',
    'full_review_text',
    'sentiment_score',
    'final_topic'
]

In [ ]:
#exporting data for dashboard
dashboard_df = new_df[columns_for_dashboard].copy()

dashboard_df.to_csv('dashboard_ready_data.csv', index=False)
print("done'dashboard_ready_data.csv'")

In [ ]:
export_df.to_excel("dashboard_ready_data.xlsx", index=False)